In [1]:
from transformers import BertTokenizer, BertForSequenceClassification ,Trainer, TrainingArguments
import csv
import pandas as pd
import os
import torch
from torch.utils.data import Dataset
from accelerate import Accelerator


/home/nelsonn/miniconda3/envs/kaleidos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-10-09 01:05:37.854978: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-09 01:05:37.858655: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 01:05:37.951680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 01:05:39.779166: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
num_labels = 2
accelerator = Accelerator()

In [3]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
sp_df = pd.read_csv('blogs_tweetfied.csv', header=None, names=['text', 'label'])
en_df = pd.read_csv('r_AnorexiaNervosa_posts.csv')

# print(sp_df['label'])
print(en_df.head)

<bound method NDFrame.head of                                                    Title  \
0                         Have some sympathy or get out.   
1                      Our Discord server is LIVE again!   
2                              What on gods name is this   
3            Am I being lazy or could this be a symptom?   
4      My group therapy has people in it that don't h...   
...                                                  ...   
13495  Going back n forth - questioning my decision t...   
13496           how long did it take you to regrow hair?   
13497                         Getting worse and worse...   
13498  My brain can’t comprehend that there are peopl...   
13499  Female Participants Wanted: ON-SITE Stanford E...   

                                                selftext  Post ID Unnamed: 3  
0      This is a post dedicated to all those that thi...  13qttgg        NaN  
1      Hello! It has been long awaited, and now it is...  152kbgw        NaN  
2      What 

In [5]:
sp_texts = sp_df['text'].tolist()
sp_labels = sp_df['label'].tolist()

In [6]:
class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, index):
        input_item = {key: value[index] for key, value in self.inputs.items()}
        label = self.labels[index]
        return input_item, label


In [7]:
inputs = tokenizer(sp_texts, return_tensors='pt', padding=True, truncation=True, max_length=256)

# Train the model on the Spanish data
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        per_device_train_batch_size=8,  # Adjust as needed
        output_dir='./results',
        evaluation_strategy="epoch",
        num_train_epochs=1,  # Adjust as needed
    ),
    train_dataset=CustomDataset(inputs, torch.tensor(sp_labels)),
)

trainer.train()

  0%|          | 0/137 [00:00<?, ?it/s]

TypeError: vars() argument must have __dict__ attribute